In [24]:
import pandas as pd
import pickle
import numpy as np
import re

# Load Education Data

In [25]:
# Load files from pickle files (see load_data.ipynb for code to download datasets)
census_df = pickle.load(open('census_df.pkl', 'rb'))
status_df = pickle.load(open('status_df.pkl', 'rb'))
grad_df = pickle.load(open('grad_df.pkl', 'rb'))

All three datasets have multiple levels of identifiers. Because we only care about school level and census tract level, we can simplify the dataframes by dropping unnecessary columns

In [26]:
census_df = census_df[['ncessch', 'school_name', 'street_location','city_location', 'state_location', 'zip_location', 'county_code',  'state_fips_geo', 'tract']]
status_df = status_df[['ncessch', 'school_name', 'street_location', 'city_location', 'state_location', 'zip_location', 'urban_centric_locale', 
                       'school_level', 'school_type', 'school_status', 'charter', 'magnet', 'shared_time', 'virtual',
                        'lunch_program', 'free_lunch', 'reduced_price_lunch', 'free_or_reduced_price_lunch']]
grad_df = grad_df[['STNAM', 'SCHNAM', 'NCESSCH', 'CATEGORY', 'COHORT', 'RATE']]

# Join Education Data (first pass)

In [27]:
# Update grad_df columns to better match the format of the other datasets
grad_df.columns = ['state', 'school_name', 'ncessch', 'cohort_cat', 'cohort_num', 'grad_rate']

In [28]:
# converting ncessch to int to match type before joining
census_df['ncessch'] = census_df['ncessch'].astype('int64')
status_df['ncessch'] = status_df['ncessch'].astype('int64')

In [29]:
# Joining the three datasets
temp_df = grad_df.merge(census_df, on='ncessch', how='left')

temp_df = temp_df.merge(status_df, on='ncessch', how='left')

temp_df.head(5)

,state,school_name_x,ncessch,cohort_cat,cohort_num,grad_rate,school_name_y,street_location_x,city_location_x,state_location_x,...,school_type,school_status,charter,magnet,shared_time,virtual,lunch_program,free_lunch,reduced_price_lunch,free_or_reduced_price_lunch
0,ALABAMA,Albertville High School,10000500871,ALL,375,94,Albertville High School,402 E McCord Ave,Albertville,AL,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
1,ALABAMA,Albertville High School,10000500871,CWD,19,GE80,Albertville High School,402 E McCord Ave,Albertville,AL,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
2,ALABAMA,Albertville High School,10000500871,ECD,114,90-94,Albertville High School,402 E McCord Ave,Albertville,AL,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
3,ALABAMA,Albertville High School,10000500871,HOM,7,LT50,Albertville High School,402 E McCord Ave,Albertville,AL,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
4,ALABAMA,Albertville High School,10000500871,LEP,67,75-79,Albertville High School,402 E McCord Ave,Albertville,AL,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,829.0,72.0,901.0


In [30]:
# Checking to see where matches didn't come through
temp_df.isnull().sum()

state                              0
school_name_x                      0
ncessch                            0
cohort_cat                         0
cohort_num                         0
grad_rate                          0
school_name_y                    357
street_location_x                357
city_location_x                  357
state_location_x                 357
zip_location_x                   357
county_code                      357
state_fips_geo                   357
tract                            357
school_name                      357
street_location_y                357
city_location_y                  357
state_location_y                 357
zip_location_y                   357
urban_centric_locale             357
school_level                     357
school_type                      357
school_status                    357
charter                          357
magnet                          1609
shared_time                     1609
virtual                         1609
l

# Address Missing Tract Matches

In [31]:
# Take a look at what schools didn't match

ids = temp_df[temp_df['tract'].isnull()]['ncessch'].unique()

no_match = []

for id in ids:
    no_match.append(grad_df[grad_df['ncessch'] == id]['school_name'].unique()[0])

print(f'There are {len(no_match)} schools missing a match')

There are 49 schools missing a match


In [32]:
# Check to see if we can find a match on name for any of the schools in either of the other datasets
census_data = []
status_data = []
not_found = []
for school in no_match:
    if len(census_df[census_df['school_name'] == school]) > 0:
        census_data.append(school)
    if len(status_df[status_df['school_name'] == school]) > 0:
        status_data.append(school)
    else:
        not_found.append(school)

print('Census:', len(census_data), '\nStatus:', len(status_data), '\nNo Match:', len(not_found))

Census: 29 
Status: 29 
No Match: 20


In [33]:
# double check that the 29 in census and status are the same ones
both_match = [i in status_data for i in census_data]

False in both_match

False

In [34]:
# Using the 29 Schools that match, build a df to compare the ncessch numbers
temp_dict = {}
count = 0
for school in census_data:
    census = [i for i in census_df[census_df['school_name'] == school]['ncessch'].unique()]
    grad = [i for i in grad_df[grad_df['school_name'] == school]['ncessch'].unique()]
    status = [i for i in status_df[status_df['school_name'] == school]['ncessch'].unique()]
    temp_dict[count] = [school, grad, census, status]
    count+= 1

temp_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=['school_name', 'grad_data', 'census_data', 'status_data'])

temp_df

,school_name,grad_data,census_data,status_data
0,Santa Cruz County Career Advancement Charter,[60202814200],[69103614200],[69103614200]
1,Pacific Collegiate Charter,[69103608363],[60254208363],[60254208363]
2,Tehama eLearning Academy,[69104310699],[60254410699],[60254410699]
3,Henryville Jr & Sr High School,[180937001531],[180021201531],[180021201531]
4,Silver Creek High School,[180937001536],"[80537001814, 180021301536]","[80537001814, 180021301536]"
5,William W Borden High School,[180937001540],[180021201540],[180021201540]
6,Kent Education CenterBeltline,[261644000888],[268062000888],[268062000888]
7,Lincoln School,[261644005328],"[171857006295, 173123003268, 180345000475, 201...","[171857006295, 173123003268, 180345000475, 201..."
8,Pine Grove Learning Center,[261644008875],[268062008875],[268062008875]
9,Lincoln Developmental Center,[261644008884],[268062008884],[268062008884]


In [35]:
# Add the 20 with no matching school name
print(len(temp_df))

for school in not_found:
    id = [i for i in grad_df[grad_df['school_name'] == school]['ncessch'].unique()]
    temp_list= [school, id, 'No Match', 'No Match']
    temp_df.loc[len(temp_df)] = temp_list

print(len(temp_df))

# Convert to integers to make things easier
temp_df['grad_data'] = temp_df['grad_data'].apply(lambda x: x[0])

temp_df

29
49


,school_name,grad_data,census_data,status_data
0,Santa Cruz County Career Advancement Charter,60202814200,[69103614200],[69103614200]
1,Pacific Collegiate Charter,69103608363,[60254208363],[60254208363]
2,Tehama eLearning Academy,69104310699,[60254410699],[60254410699]
3,Henryville Jr & Sr High School,180937001531,[180021201531],[180021201531]
4,Silver Creek High School,180937001536,"[80537001814, 180021301536]","[80537001814, 180021301536]"
5,William W Borden High School,180937001540,[180021201540],[180021201540]
6,Kent Education CenterBeltline,261644000888,[268062000888],[268062000888]
7,Lincoln School,261644005328,"[171857006295, 173123003268, 180345000475, 201...","[171857006295, 173123003268, 180345000475, 201..."
8,Pine Grove Learning Center,261644008875,[268062008875],[268062008875]
9,Lincoln Developmental Center,261644008884,[268062008884],[268062008884]


In order to determine the correct ncessch ID, if one existed, I used the nces.ed.gov site to look up the schools. the ncessch IDs are made up of 3 parts a state code, a district code, and a school ID. In almost all cases it appeared that schools had moved to a new district so while the state and school ID were the same, teh district ID had changed causing the mismatch. For instance Edisto High had (45)(03180)(00862) as its original ID, but after redistricting changed to (45)(03910)(00862). Using this structure I was able to identify the matching items in the census and status datsets. 

In the event that no match could be found using the school name or any of the avaialble ncessch IDs I marked it with 'No Match'

In [36]:
# Dictionary of the correct ncessch IDs
temp_dict = {0: 69103614200, 1: 60254208363, 2: 60254410699, 3: 180021201531, 4: 180021301536, 5: 180021201540, 6: 268062000888, 7: 268062005328, 
            8: 268062008875, 9: 268062008884, 10: 268085007799, 11: 360009105927, 12: 360009505994, 13: 410363001756, 14: 450391001192, 15: 450391000196, 
            16: 450391000862, 17: 450391000880, 18: 450391000850, 19: 450391000873 , 20: 450391000876, 21: 'No Match', 22: 500044300210, 
            23: 500044800207, 24: 500044000127, 25: 500044400022, 26: 500044400256, 27: 'No Match', 28: 500044200539, 29: 'No Match', 30: 'No Match',
            31: 'No Match', 32: 'No Match', 33: 'No Match', 34: 'No Match', 35: 'No Match', 36: 'No Match', 37: 'No Match', 38: 'No Match', 39: 'No Match',
            40: 'No Match', 41: 'No Match', 42: 'No Match', 43: 500044500419, 44: 500043100051, 45: 500043200173, 46: 500043300195, 47: 500043900339, 
            48: 500043709272}

In [37]:
# Update grad_df with new ncessch IDs
def ncessch_match(row):
    '''Function to be used with apply on grad_Df. Used to identify rows with ncessch IDs that didn't have a match and then update to the correct ncessch ID'''
    old_id = row['ncessch']
    if temp_df['grad_data'].isin([old_id]).any():
        ind = temp_df[temp_df['grad_data'] == old_id].index[0]
        new_id = temp_dict[ind]
        return new_id
    else: return old_id

grad_df['ncessch'] = grad_df.apply(ncessch_match, axis=1)

In [38]:
# Spot checking a few schools to make sure the ncessch ID updated
print(grad_df[grad_df['school_name'] == 'Windsor School'])
print(grad_df[grad_df['school_name'] == 'Malan\'s Peak Secondary'])
print(grad_df[grad_df['school_name'] == 'Missisquoi Valley UHSD #7'])

          state     school_name       ncessch cohort_cat  cohort_num grad_rate
177228  VERMONT  Windsor School  500044200539        ALL          59     80-89
177229  VERMONT  Windsor School  500044200539        CWD           8      GE50
177230  VERMONT  Windsor School  500044200539        ECD          22     60-79
177231  VERMONT  Windsor School  500044200539        HOM           3        PS
177232  VERMONT  Windsor School  500044200539        MTR           1        PS
177233  VERMONT  Windsor School  500044200539        MWH          57     70-79
       state             school_name   ncessch cohort_cat  cohort_num  \
176360  UTAH  Malan's Peak Secondary  No Match        ALL          21   
176361  UTAH  Malan's Peak Secondary  No Match        CWD           6   
176362  UTAH  Malan's Peak Secondary  No Match        ECD           5   
176363  UTAH  Malan's Peak Secondary  No Match        MBL           5   
176364  UTAH  Malan's Peak Secondary  No Match        MHI           7   
176365  U

In [39]:
# Dropping all schools that I could not find a match for
print('Number of rows with no match:', len(grad_df[grad_df['ncessch'] == 'No Match']))
print('Length of Dataframe before dropping:', len(grad_df))

grad_df = grad_df[grad_df['ncessch'] != 'No Match']

print('Length of DataFrame after filetering:', len(grad_df))

Number of rows with no match: 105
Length of Dataframe before dropping: 193753
Length of DataFrame after filetering: 193648


# Join Education Data (second pass)

In [40]:
# Convert ncessch back to an integer
grad_df['ncessch'] = grad_df['ncessch'].astype('int64')

In [41]:
edu_df = grad_df.merge(census_df, on='ncessch', how='left')

edu_df = edu_df.merge(status_df, on='ncessch', how='left')

edu_df.head(5)

,state,school_name_x,ncessch,cohort_cat,cohort_num,grad_rate,school_name_y,street_location_x,city_location_x,state_location_x,...,school_type,school_status,charter,magnet,shared_time,virtual,lunch_program,free_lunch,reduced_price_lunch,free_or_reduced_price_lunch
0,ALABAMA,Albertville High School,10000500871,ALL,375,94,Albertville High School,402 E McCord Ave,Albertville,AL,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
1,ALABAMA,Albertville High School,10000500871,CWD,19,GE80,Albertville High School,402 E McCord Ave,Albertville,AL,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
2,ALABAMA,Albertville High School,10000500871,ECD,114,90-94,Albertville High School,402 E McCord Ave,Albertville,AL,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
3,ALABAMA,Albertville High School,10000500871,HOM,7,LT50,Albertville High School,402 E McCord Ave,Albertville,AL,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
4,ALABAMA,Albertville High School,10000500871,LEP,67,75-79,Albertville High School,402 E McCord Ave,Albertville,AL,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0


We have now sucessfully matched all of the schools in grad_df with their associated school in census_df and status_df. There are still missing values from the status_df dataset, but since we'll be using those primarily for filtering we don't need to worry about those yet. 

In [42]:
# Check if any mismatches remain

edu_df.isnull().sum()

state                              0
school_name_x                      0
ncessch                            0
cohort_cat                         0
cohort_num                         0
grad_rate                          0
school_name_y                      0
street_location_x                  0
city_location_x                    0
state_location_x                   0
zip_location_x                     0
county_code                        0
state_fips_geo                     0
tract                              0
school_name                        0
street_location_y                  0
city_location_y                    0
state_location_y                   0
zip_location_y                     0
urban_centric_locale               0
school_level                       0
school_type                        0
school_status                      0
charter                            0
magnet                          1252
shared_time                     1252
virtual                         1252
l

# Filter Dataset Down

In [43]:
edu_df.columns

Index(['state', 'school_name_x', 'ncessch', 'cohort_cat', 'cohort_num',
       'grad_rate', 'school_name_y', 'street_location_x', 'city_location_x',
       'state_location_x', 'zip_location_x', 'county_code', 'state_fips_geo',
       'tract', 'school_name', 'street_location_y', 'city_location_y',
       'state_location_y', 'zip_location_y', 'urban_centric_locale',
       'school_level', 'school_type', 'school_status', 'charter', 'magnet',
       'shared_time', 'virtual', 'lunch_program', 'free_lunch',
       'reduced_price_lunch', 'free_or_reduced_price_lunch'],
      dtype='object')

In [44]:
# Drop duplicated columns

edu_df = edu_df.drop(columns=['state', 'school_name_y', 'school_name', 'street_location_y', 
                            'city_location_y', 'state_location_y','zip_location_y'])

In [45]:
# Drop suffix from columns

edu_df.rename(columns= {'school_name_x' : 'school_name', 'street_location_x': 'street',  'state_location_x': 'state', 
                        'city_location_x': 'city', 'zip_location_x': 'zip', 'state_fips_geo': 'state_code'}, inplace= True)

In [46]:
edu_df.head()

,school_name,ncessch,cohort_cat,cohort_num,grad_rate,street,city,state,zip,county_code,...,school_type,school_status,charter,magnet,shared_time,virtual,lunch_program,free_lunch,reduced_price_lunch,free_or_reduced_price_lunch
0,Albertville High School,10000500871,ALL,375,94,402 E McCord Ave,Albertville,AL,35950,1095,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
1,Albertville High School,10000500871,CWD,19,GE80,402 E McCord Ave,Albertville,AL,35950,1095,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
2,Albertville High School,10000500871,ECD,114,90-94,402 E McCord Ave,Albertville,AL,35950,1095,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
3,Albertville High School,10000500871,HOM,7,LT50,402 E McCord Ave,Albertville,AL,35950,1095,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0
4,Albertville High School,10000500871,LEP,67,75-79,402 E McCord Ave,Albertville,AL,35950,1095,...,1,1,0,0.0,0.0,0.0,2.0,829.0,72.0,901.0


Now that we have the schools all properly matched, we'll filter out some of the schools to try and get a population that is relatively similar and representative of the population we want to look at. We'll do this using the following parameters:

School Type: Exclude special education, vocational, and other/alternative schools as well as reportable programs since these programs would have a number of factors that would impact graduation rates that 'Regular Schools' would not. (Only include schools with 1 or < 0)

School Status: Exclude schools that were closed (2) or inactive (6) at the beginning of the school year since the timeline for graduation rates would be different than schools open during the entire school year.

Exclude: Magnet Schools, Shared Time, and Virtual Schools since the student population of these schools may not live in the area their school is located and therefor may have different food access. A note on these that there are a number of schools where this information is not reported (-1) or not applicable (-2), because these types of schools are less common and I don't have an easy way to assess where they fall I will be assuming that this information missing is equivalent to a school not being part of that category and keep them in the dataset.


In [47]:
# Drop schools with unwanted types
edu_df = edu_df[(edu_df['school_type'] < 2) & (edu_df['school_status'] != 2) & (edu_df['school_status'] != 6) &
                (edu_df['magnet'] != 1.0) & (edu_df['shared_time'] != 1.0) & (edu_df['virtual'] != 1.0) & (edu_df['virtual'] != 2)]

In [48]:
# Check new values in each column
cols = ['school_type', 'school_status', 'magnet', 'shared_time', 'virtual']

for col in cols:
    print(col, ':', edu_df[col].unique())

school_type : [1]
school_status : [1 5]
magnet : [ 0. -1. -2.]
shared_time : [ 0. -1.]
virtual : [ 0.  3. -1.]


In [49]:
# Check how many unique schools are left after filtering
len(edu_df['ncessch'].unique())

17257

In [50]:
edu_df.isnull().sum()

school_name                        0
ncessch                            0
cohort_cat                         0
cohort_num                         0
grad_rate                          0
street                             0
city                               0
state                              0
zip                                0
county_code                        0
state_code                         0
tract                              0
urban_centric_locale               0
school_level                       0
school_type                        0
school_status                      0
charter                            0
magnet                             0
shared_time                        0
virtual                            0
lunch_program                      0
free_lunch                     14644
reduced_price_lunch            14644
free_or_reduced_price_lunch     9361
dtype: int64

I was originally hoping to use the number of students on free or reduced lunch as a feature, but there are so many missing values it doesn't look like this will be useful. I may still be able to use the 'lunch_program' (if/how the school participates in the National School Lunch Program) as a feature though.

I will drop those three columns, along with the columns I used to filter since I will no longer need those.

In [51]:
edu_df.drop(columns=['school_type', 'school_status', 'magnet', 'shared_time', 'virtual', 'free_lunch', 'reduced_price_lunch', 'free_or_reduced_price_lunch'], inplace=True)

# Explore remaining data

We've now cleared out all the missing values and filtered our data down to the features we want to consider. Now we will explore this data some more.

In [52]:
# Take a look at remaining features
edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149281 entries, 0 to 193647
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   school_name           149281 non-null  object 
 1   ncessch               149281 non-null  int64  
 2   cohort_cat            149281 non-null  object 
 3   cohort_num            149281 non-null  int64  
 4   grad_rate             149281 non-null  object 
 5   street                149281 non-null  object 
 6   city                  149281 non-null  object 
 7   state                 149281 non-null  object 
 8   zip                   149281 non-null  object 
 9   county_code           149281 non-null  int64  
 10  state_code            149281 non-null  int64  
 11  tract                 149281 non-null  float64
 12  urban_centric_locale  149281 non-null  int64  
 13  school_level          149281 non-null  int64  
 14  charter               149281 non-null  int64  
 15  

# Clean Up Grad Rate

There are 4 types of notation for Grad Rate that need to be addressed:

1) A single number: This means that the cohort was large enough that the true graduation rate was reported, we cna just use this number.
2) Two numbers with a dash: This means that the graduation rate was given as a range to protect privacy. For these, we'll take the average of the two numbers as teh graduation rate
3) GE/LE: This means greater/less than or equal to the number provided. For these, since we have no way of getting more accurate numbers, we'll just take the number provided as the rate
4) PS: This means the cohort was small enough that the graduation rate was surpressed. We'll need to determine whether these can be dropped or if we need to impute these

In [53]:
# Look at the values of grad rate

edu_df['grad_rate'].unique()


array(['94', 'GE80', '90-94', 'LT50', '75-79', 'GE50', 'PS', 'GE95',
       'GE90', '85-89', '80-89', '93', '50-59', '70-79', '97', '80-84',
       '40-59', '98', '70-74', '91', '60-79', '60-64', '65-69', '95',
       '96', '90', '88', '92', '60-69', '87', '84', '78', '21-39', '80',
       'LE20', '89', '85', '86', '40-49', '81', '82', '83', '55-59',
       '45-49', '74', '30-34', '20-29', '40-44', '35-39', '61', '77',
       '79', '76', '30-39', 'GE99', '75', '73', '11-19', '68', '57',
       '50-54', '71', '72', '25-29', '20-24', '20', '10-14', '18',
       '15-19', 'LE5', 'LE10', '2', 'LE1', '14', '6-9', '13', '11', '19',
       '17', '31', '29', '10', '9', '16', '22', '21', '41', '37', '58',
       '24', '23', '5', '8', '4', '3', '28', '12', '15', '70', '69', '66',
       '59', '62', '56', '26', '65', '54', '64', '63', '67', '53', '35',
       '39', '60', '49', '52', '40', '48', '25', '33', '55', '43', '27',
       '46', '45', '44'], dtype=object)

In [54]:
# Pull out a graduation rate for all observations
def grad_rate(rate):
    '''Function to assess the string notation for grad rate and convert to a single number or null value if data is supressed'''
    if 'PS' in rate:
        return np.nan
    if 'GE' in rate or 'LE' in rate:
        new_rate = re.sub(r'\S["E"]', '', rate)
        return new_rate
    if 'GT' in rate or 'LT' in rate:
        new_rate = re.sub(r'\S["T"]', '', rate)
        return new_rate
    if '-' in rate:
        nums = rate.split('-')
        new_rate = (int(nums[0]) + int(nums[1]))/2
        return new_rate
    else:
        return rate
    
edu_df['grad_num'] = edu_df['grad_rate'].apply(grad_rate)

In [55]:
edu_df['grad_num'].unique()

array(['94', '80', 92.0, '50', 77.0, nan, '95', '90', 87.0, 84.5, '93',
       54.5, 74.5, '97', 82.0, 49.5, '98', 72.0, '91', 69.5, 62.0, 67.0,
       '96', '88', '92', 64.5, '87', '84', '78', 30.0, '20', '89', '85',
       '86', 44.5, '81', '82', '83', 57.0, 47.0, '74', 32.0, 24.5, 42.0,
       37.0, '61', '77', '79', '76', 34.5, '99', '75', '73', 15.0, '68',
       '57', 52.0, '71', '72', 27.0, 22.0, 12.0, '18', 17.0, '5', '10',
       '2', '1', '14', 7.5, '13', '11', '19', '17', '31', '29', '9', '16',
       '22', '21', '41', '37', '58', '24', '23', '8', '4', '3', '28',
       '12', '15', '70', '69', '66', '59', '62', '56', '26', '65', '54',
       '64', '63', '67', '53', '35', '39', '60', '49', '52', '40', '48',
       '25', '33', '55', '43', '27', '46', '45', '44'], dtype=object)

In [56]:
# Check how many observations are missing values
edu_df.isnull().sum()

school_name                 0
ncessch                     0
cohort_cat                  0
cohort_num                  0
grad_rate                   0
street                      0
city                        0
state                       0
zip                         0
county_code                 0
state_code                  0
tract                       0
urban_centric_locale        0
school_level                0
charter                     0
lunch_program               0
grad_num                50899
dtype: int64

There are enough missing values for grad rates that we will want to impute a value rather than drop them. In order to create some sort of similarity, rather than just use the average graduation rate, we will use the average graduation rates for students in the same state, urban centric locale, and sub_cohort since these populations are most likely to be similar in other respects. 

First we'll need to get urban_centric_locale in a usable state. We'll do this by grouping the provided numbers into the top level categories

Source:  https://nces.ed.gov/surveys/annualreports/topical-studies/locale/definitions

In [57]:
# First we'll clean up urban centric locale

# Create urban centric locale category
def locale (x):
    if x['urban_centric_locale'] in [1, 2, 3, 4, 11, 12, 13]:
        return 'Urban'
    if x['urban_centric_locale'] in [21, 22, 23]:
        return 'Suburban'
    if x['urban_centric_locale'] in [5, 6, 31, 32, 33]:
        return 'Town'
    if x['urban_centric_locale'] in [7, 8, 41, 42, 43]:
        return 'Rural'
    # Returning a -1 will catch any values that don't match the definitions
    else:
        return -1
    
edu_df['urban_locale_cat'] = edu_df.apply(locale, axis='columns')

In [58]:
# Double check that all values updated
edu_df['urban_locale_cat'].unique()

array(['Town', 'Rural', 'Urban', 'Suburban'], dtype=object)

In [59]:
# Create a pivot table of graduation rates by state, urban locale, and cohort
temp_df = edu_df.dropna(subset='grad_num')

temp_df['grad_num'] = temp_df['grad_num'].astype('int32')

avg_grad = temp_df.pivot_table(index=['state', 'urban_locale_cat', 'cohort_cat'], values='grad_num', aggfunc=lambda x: round(np.mean(x), 2))

avg_grad

C:\Users\bcbot\AppData\Local\Temp\ipykernel_64468\237273259.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['grad_num'] = temp_df['grad_num'].astype('int32')


grad_num
state urban_locale_cat cohort_cat          
AK    Rural            ALL            59.00
                       CWD            47.50
                       ECD            55.48
                       HOM            50.00
                       LEP            49.93
...                                     ...
WY    Urban            MAS            50.00
                       MBL            50.00
                       MHI            68.86
                       MTR            50.00
                       MWH            73.29

[2101 rows x 1 columns]

In [60]:
def grad_rate_fill_na (row):
    if pd.isnull(row['grad_num']) == True:
        state = row['state']
        locale = row['urban_locale_cat']
        cohort = row['cohort_cat']

        if cohort in avg_grad.loc[state].loc[locale].index:
            return avg_grad.loc[state].loc[locale].loc[cohort]['grad_num']
        # This occurs if there are no cohorts of that type anywhere in that state/locale combination who had a graduation rate reported
        else:
            return -1
    else:
        return row['grad_num']
        

edu_df['grad_num_2'] = edu_df.apply(grad_rate_fill_na, axis=1)

Let's look at what that did for us. It looks like we managed to impute a value for over 47,000 of our observations and are no left with only 3,150 observations with not graduation rate. These occured because that particular group is so small and/or spread out that there is no instance within that state/locale combination of that particular cohort category that was large enough to have their graduation rates reported.

On the plus side, there are no schools that don't at least have a graduation rate reported for their total student body. 

At this point, any values we impute for the reaminin 3k observations would be random guesses, and these groups represent only a tiny fraction of the student body, so it is probably not going to add much to our analysis. So at this point, I will drop anything that we were unable to impute a value for. 

In [61]:
print('Prior to imputing values, the number of observations missing a graduation rate was:', len(edu_df[edu_df['grad_num'].isnull()]))

print('After imputing values, the number of observations missing a graduation rate was: ', len(edu_df[edu_df['grad_num_2'] == -1]))

print('The total number of schools with no graduation rates availalbe for the total cohort was: ', len(edu_df[(edu_df['grad_num_2'] == -1) & (edu_df['cohort_cat'] == 'ALL')]))

Prior to imputing values, the number of observations missing a graduation rate was: 50899
After imputing values, the number of observations missing a graduation rate was:  3150
The total number of schools with no graduation rates availalbe for the total cohort was:  0


In [62]:
edu_df = edu_df[edu_df['grad_num_2'] != -1]

The cohort categories are a little hard to follow since they use custom coding, so lets update these to be easier to read. 

In [63]:
cohort_dict = {'ALL': 'total', 'MAM': 'am_indian/ak_native', 'MAS': 'api', 'MHI': 'hispanic', 'MBL': 'black', 'MWH': 'white', 'MTR': 'two_or_more', "CWD": 'disability',
               'ECD': 'econ_disadvantaged', 'LEP': 'limited_english', 'HOM': 'homeless', 'FCS': 'foster_care'}

edu_df['cohort_cat'] = edu_df['cohort_cat'].apply(lambda cat: cohort_dict[cat])

edu_df

,school_name,ncessch,cohort_cat,cohort_num,grad_rate,street,city,state,zip,county_code,state_code,tract,urban_centric_locale,school_level,charter,lunch_program,grad_num,urban_locale_cat,grad_num_2
0,Albertville High School,10000500871,total,375,94,402 E McCord Ave,Albertville,AL,35950,1095,1,30902.0,32,3,0,2.0,94,Town,94
1,Albertville High School,10000500871,disability,19,GE80,402 E McCord Ave,Albertville,AL,35950,1095,1,30902.0,32,3,0,2.0,80,Town,80
2,Albertville High School,10000500871,econ_disadvantaged,114,90-94,402 E McCord Ave,Albertville,AL,35950,1095,1,30902.0,32,3,0,2.0,92.0,Town,92.0
3,Albertville High School,10000500871,homeless,7,LT50,402 E McCord Ave,Albertville,AL,35950,1095,1,30902.0,32,3,0,2.0,50,Town,50
4,Albertville High School,10000500871,limited_english,67,75-79,402 E McCord Ave,Albertville,AL,35950,1095,1,30902.0,32,3,0,2.0,77.0,Town,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193643,JUAN QUIRINDONGO MORELL (SUPERIOR),720003002085,total,182,80-84,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,72,560701.0,21,3,0,1.0,82.0,Suburban,82.0
193644,JUAN QUIRINDONGO MORELL (SUPERIOR),720003002085,disability,36,60-69,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,72,560701.0,21,3,0,1.0,64.5,Suburban,64.5
193645,JUAN QUIRINDONGO MORELL (SUPERIOR),720003002085,econ_disadvantaged,146,80-84,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,72,560701.0,21,3,0,1.0,82.0,Suburban,82.0
193646,JUAN QUIRINDONGO MORELL (SUPERIOR),720003002085,homeless,1,PS,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,72,560701.0,21,3,0,1.0,NaN,Suburban,50.0


In [64]:
# Drop columns no longer needed. 
edu_df.drop(columns=['grad_rate', 'urban_centric_locale', 'grad_num'], inplace=True)

In [65]:
edu_df.rename(columns={'grad_num_2': 'grad_rate'}, inplace=True)

edu_df['grad_rate'] = edu_df['grad_rate'].astype('int64')
edu_df['urban_locale_cat'] = edu_df['urban_locale_cat'].astype('category')

In [66]:
edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146131 entries, 0 to 193647
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   school_name       146131 non-null  object  
 1   ncessch           146131 non-null  int64   
 2   cohort_cat        146131 non-null  object  
 3   cohort_num        146131 non-null  int64   
 4   street            146131 non-null  object  
 5   city              146131 non-null  object  
 6   state             146131 non-null  object  
 7   zip               146131 non-null  object  
 8   county_code       146131 non-null  int64   
 9   state_code        146131 non-null  int64   
 10  tract             146131 non-null  float64 
 11  school_level      146131 non-null  int64   
 12  charter           146131 non-null  int64   
 13  lunch_program     146131 non-null  float64 
 14  urban_locale_cat  146131 non-null  category
 15  grad_rate         146131 non-null  int64   
dtypes:

In [67]:
edu_df.sample(10)

,school_name,ncessch,cohort_cat,cohort_num,street,city,state,zip,county_code,state_code,tract,school_level,charter,lunch_program,urban_locale_cat,grad_rate
149631,Pen Argyl Area HS,421857006439,hispanic,4,501 W Laurel Ave,Pen Argyl,PA,18072,42095,42,15600.0,3,0,1.0,Suburban,68
5052,American Leadership Academy - Queen Creek,40011203179,econ_disadvantaged,28,22721 S ELLSWORTH RD STE 103,QUEEN CREEK,AZ,85142,4013,4,817200.0,4,1,0.0,Suburban,80
58170,Maui High School,150003000073,disability,45,660 S Lono Ave,Kahului,HI,96732,15009,15,31103.0,3,0,1.0,Urban,64
172380,ROCHELLE SCHOOL,483750004212,total,10,5902 LAFAYETTE,ROCHELLE,TX,76872,48307,48,950500.0,4,0,2.0,Rural,50
143711,Marshfield Senior High School,410366000216,total,180,10th & Ingersoll,Coos Bay,OR,97420,41011,41,700.0,3,0,1.0,Town,92
148636,Keystone Oaks HS,421275005112,api,6,1000 Kelton Ave,Pittsburgh,PA,15216,42003,42,473300.0,3,0,1.0,Suburban,50
189655,WHS Information Technology,550960002608,two_or_more,5,2525 N Sherman Blvd,Milwaukee,WI,53210,55079,55,6100.0,3,0,2.0,Urban,58
153354,Dillon High,450192000385,hispanic,4,1730 Highway 301 North,Dillon,SC,29536,45033,45,970300.0,3,0,2.0,Town,64
191186,Waterloo High,551572001995,foster_care,1,865 N Monroe St,Waterloo,WI,53594,55055,55,100400.0,3,0,1.0,Rural,50
141830,PRESTON HS,402505001315,white,18,10061 OLD HWY 75,Preston,OK,74456,40111,40,500.0,3,0,1.0,Rural,80


In [68]:
# Save final file as a pickle file

edu_df.to_pickle(open('edu_df.pkl', 'wb'))